# Moving and Rotating a Molecular Cluster:
**Task:** 
1. Move/rotate a Molecule/Atom one by one systematically
2. Move/Rotate randomly

## Water molecule ($H_{2}O$):

Cartesian coordinates [Angstrom]:

    
    O    0.00000    0.00000    0.00000
    H    0.58708    0.75754    0.00000
    H    0.58708   -0.75754    0.00000


> **_NOTE:_**   
> `translate` and `rotate` methods return a brand new Molecule object

In [1]:
# creating the path (PYTHONPATH) to our module.
# assuming that our 'amcess' directory is out ('..') of our current directory 
import os
import sys
module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
# importing de Molecule Class
from amcess.base_molecule import Molecule, Cluster

In [3]:
# First, let's create a Molecule by using the atomic constituents
# coordinates (use list or dictionary, see notebook: 01)

water = [("O", 0, 0, 0), ("H", 0.58708, 0.75754, 0), ("H", -0.58708, 0.75754, 0)]

water_molecule = Molecule(water)

print(water_molecule.xyz)

	3
-- charge=0 and multiplicity=1 --
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	     0.00000000
H     	    -0.58708000	     0.75754000	     0.00000000



In [4]:
# Now, let's create a Cluster by adding three water molecules with the 
# same cartesian coordinates. 
water3 = Cluster(water_molecule + water_molecule + water_molecule)

# which is equivalent to 
water3 = Cluster(3 * water_molecule)

print(water3)
print(water3.xyz)

Cluster of (3) molecules and (9) total atoms
 #0: molecule with 3 atoms:
     --> atoms: [('O', 0, 0, 0), ('H', 0.58708, 0.75754, 0), ('H', -0.58708, 0.75754, 0)]
     --> charge: +0
     --> multiplicity: 1
 #1: molecule with 3 atoms:
     --> atoms: [('O', 0, 0, 0), ('H', 0.58708, 0.75754, 0), ('H', -0.58708, 0.75754, 0)]
     --> charge: +0
     --> multiplicity: 1
 #2: molecule with 3 atoms:
     --> atoms: [('O', 0, 0, 0), ('H', 0.58708, 0.75754, 0), ('H', -0.58708, 0.75754, 0)]
     --> charge: +0
     --> multiplicity: 1

	9
-- charge=0 and multiplicity=1 --
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	     0.00000000
H     	    -0.58708000	     0.75754000	     0.00000000
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	     0.00000000
H     	    -0.58708000	     0.75754000	     0.00000000
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	    

In [9]:
# We can also create a cluster by using the coordinates of the atomic constituents (list of tuples/dictionary)
water3 = Cluster(water, water, water)

print(water3)
print(water3.xyz)

Cluster of (3) molecules and (9) total atoms
 #0: molecule with 3 atoms:
     --> atoms: [('O', 0, 0, 0), ('H', 0.58708, 0.75754, 0), ('H', -0.58708, 0.75754, 0)]
     --> charge: +0
     --> multiplicity: 1
 #1: molecule with 3 atoms:
     --> atoms: [('O', 0, 0, 0), ('H', 0.58708, 0.75754, 0), ('H', -0.58708, 0.75754, 0)]
     --> charge: +0
     --> multiplicity: 1
 #2: molecule with 3 atoms:
     --> atoms: [('O', 0, 0, 0), ('H', 0.58708, 0.75754, 0), ('H', -0.58708, 0.75754, 0)]
     --> charge: +0
     --> multiplicity: 1

	9
-- charge=0 and multiplicity=1 --
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	     0.00000000
H     	    -0.58708000	     0.75754000	     0.00000000
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	     0.00000000
H     	    -0.58708000	     0.75754000	     0.00000000
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	    

In [10]:
# Additionally, you can mix both formalisms (list/dictionary and molecule object)
water3 = Cluster(water, 2 * water_molecule)

print(water3)
print(water3.xyz)

Cluster of (3) molecules and (9) total atoms
 #0: molecule with 3 atoms:
     --> atoms: [('O', 0, 0, 0), ('H', 0.58708, 0.75754, 0), ('H', -0.58708, 0.75754, 0)]
     --> charge: +0
     --> multiplicity: 1
 #1: molecule with 3 atoms:
     --> atoms: [('O', 0, 0, 0), ('H', 0.58708, 0.75754, 0), ('H', -0.58708, 0.75754, 0)]
     --> charge: +0
     --> multiplicity: 1
 #2: molecule with 3 atoms:
     --> atoms: [('O', 0, 0, 0), ('H', 0.58708, 0.75754, 0), ('H', -0.58708, 0.75754, 0)]
     --> charge: +0
     --> multiplicity: 1

	9
-- charge=0 and multiplicity=1 --
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	     0.00000000
H     	    -0.58708000	     0.75754000	     0.00000000
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	     0.00000000
H     	    -0.58708000	     0.75754000	     0.00000000
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	    

>### NOTE:
>Creating a `Molecule` object means that its atoms are FIXED and 
>those atoms **can NOT** be MOVED or ROTATED.
>
>On the other hand, creating a `Cluster` object means their individual
>molecules **can** be MOVED or ROTATED

In [5]:
# Let's create a cluster with different species: five (5) water molecules and one atom (Iodine)
iodine = [("I", 0, 0, 0)]

# We can do so by using the coordinates of each specie
w5_I = Cluster(
    iodine,
    water,
    water,
    water,
    water,
    water,
)

# which is equivalent to the following expressions
w5_I = Cluster(iodine, 5 * water_molecule)
w5_I = Cluster(Molecule(iodine) + 5 * water_molecule)

# Remember you will ALWAYS have SIX (6) individual molecules/atoms in your 'w5_I' cluster
print(w5_I)

Cluster of (6) molecules and (16) total atoms
 #0: molecule with 1 atoms:
     --> atoms: [('I', 0, 0, 0)]
     --> charge: +0
     --> multiplicity: 1
 #1: molecule with 3 atoms:
     --> atoms: [('O', 0, 0, 0), ('H', 0.58708, 0.75754, 0), ('H', -0.58708, 0.75754, 0)]
     --> charge: +0
     --> multiplicity: 1
 #2: molecule with 3 atoms:
     --> atoms: [('O', 0, 0, 0), ('H', 0.58708, 0.75754, 0), ('H', -0.58708, 0.75754, 0)]
     --> charge: +0
     --> multiplicity: 1
 #3: molecule with 3 atoms:
     --> atoms: [('O', 0, 0, 0), ('H', 0.58708, 0.75754, 0), ('H', -0.58708, 0.75754, 0)]
     --> charge: +0
     --> multiplicity: 1
 #4: molecule with 3 atoms:
     --> atoms: [('O', 0, 0, 0), ('H', 0.58708, 0.75754, 0), ('H', -0.58708, 0.75754, 0)]
     --> charge: +0
     --> multiplicity: 1
 #5: molecule with 3 atoms:
     --> atoms: [('O', 0, 0, 0), ('H', 0.58708, 0.75754, 0), ('H', -0.58708, 0.75754, 0)]
     --> charge: +0
     --> multiplicity: 1



In [12]:
# asking for its properties
print(f"\nNumber of individual molecules/atoms: \n{w5_I.total_molecules}")
print(f"\nTotal number of atoms: \n{w5_I.total_atoms}")
print(f"\nAtomic symbols: \n{w5_I.symbols}")
print(f"\nIndividual atomic masses: \n{w5_I.atomic_masses}")
print(f"\nMolecular mass: \n{w5_I.total_mass}")
print(f"\nCoordinates: \n{w5_I.atoms}")
print(f"\nCartesian coordinates: \n{w5_I.coordinates}")


Number of individual molecules/atoms: 
6

Total number of atoms: 
16

Atomic symbols: 
['I', 'O', 'H', 'H', 'O', 'H', 'H', 'O', 'H', 'H', 'O', 'H', 'H', 'O', 'H', 'H']

Individual atomic masses: 
[126.9045, 15.999, 1.008, 1.008, 15.999, 1.008, 1.008, 15.999, 1.008, 1.008, 15.999, 1.008, 1.008, 15.999, 1.008, 1.008]

Molecular mass: 
216.9795000000001

Coordinates: 
[('I', 0, 0, 0), ('O', 0, 0, 0), ('H', 0.58708, 0.75754, 0), ('H', -0.58708, 0.75754, 0), ('O', 0, 0, 0), ('H', 0.58708, 0.75754, 0), ('H', -0.58708, 0.75754, 0), ('O', 0, 0, 0), ('H', 0.58708, 0.75754, 0), ('H', -0.58708, 0.75754, 0), ('O', 0, 0, 0), ('H', 0.58708, 0.75754, 0), ('H', -0.58708, 0.75754, 0), ('O', 0, 0, 0), ('H', 0.58708, 0.75754, 0), ('H', -0.58708, 0.75754, 0)]

Cartesian coordinates: 
[(0, 0, 0), (0, 0, 0), (0.58708, 0.75754, 0), (-0.58708, 0.75754, 0), (0, 0, 0), (0.58708, 0.75754, 0), (-0.58708, 0.75754, 0), (0, 0, 0), (0.58708, 0.75754, 0), (-0.58708, 0.75754, 0), (0, 0, 0), (0.58708, 0.75754, 0), (-0.

In [13]:
# Let's MOVE 2 units on xy axes the first molecule (I) and rotate 180 deg around z-axis the second molecule (water)
w5_I_moved_rotated = w5_I.translate(0, x=2, y=2, z=0).rotate(1, x=0, y=0, z=180)

print(w5_I_moved_rotated.xyz)

	16
-- charge=0 and multiplicity=1 --
I     	     2.00000000	     2.00000000	     0.00000000
O     	    -0.00000000	     0.16954767	     0.00000000
H     	    -0.58708000	    -0.58799233	     0.00000000
H     	     0.58708000	    -0.58799233	     0.00000000
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	     0.00000000
H     	    -0.58708000	     0.75754000	     0.00000000
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	     0.00000000
H     	    -0.58708000	     0.75754000	     0.00000000
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	     0.00000000
H     	    -0.58708000	     0.75754000	     0.00000000
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	     0.00000000
H     	    -0.58708000	     0.75754000	     0.00000000



### IMPORTANT FACTS:
- After translating a molecule, a new object is created where the moved 
molecule in the same position. The same is true for any rotation
- Rotation are around the center of mass of the target molecule

In [14]:
# let's visualize its random process FREEZING Iodine
from copy import deepcopy

# creating a brand new copy when 'I' will NOT be moved or rotated 
w5 = deepcopy(w5_I)

print(w5.sphere_radius)
print(w5.sphere_center)

# setting a sphere radius for the Cluster spherical boundary conditions
w5.sphere_radius = 10

print("\nnew sphere radius")
print(w5.sphere_radius)
print(w5.sphere_center)

None
(0, 0, 0)

new sphere radius
10
(0, 0, 0)


!pip install py3Dmol
import py3Dmol

In [15]:
from numpy import random
from copy import deepcopy

# choosing a fixed seed
random_gen = random.default_rng(1234)

# to save snapshot and show as a movie
w5_xyz = ""

total_steps = 1000

# max step size to move a molecule 
step = w5.sphere_radius

for i in range(total_steps):
    # molecule to be selected between [0, w.total_molecules]
    mol = random_gen.choice(w5.total_molecules)

    # angle between [0, 360)
    ax = random_gen.uniform() * 360
    ay = random_gen.uniform() * 360
    az = random_gen.uniform() * 360

    # moving between [-step, +step]
    tx = step * (random_gen.uniform() - 0.5)
    ty = step * (random_gen.uniform() - 0.5)
    tz = step * (random_gen.uniform() - 0.5)

    # saving coordinates as a string
    # w5_xyz += w5.xyz
    if i % 100 == 0: 
        w5_xyz += w5.xyz

    w5 = w5.translate(mol, x=tx, y=ty, z=tz).rotate(
        mol, x=ax, y=ay, z=az
    )
    # printing current step
    print(
        f"\r progress {100.0*((i + 1)/total_steps):.2f}"
        f" % -- step {i + 1}/{total_steps}", end=''
        )

# -------------------------------------------------------
print("\n *** JOB DONE ***")
print(f"after {total_steps} steps\n")
print(w5.xyz)

 progress 100.00 % -- step 1000/1000
 *** JOB DONE ***
after 1000 steps

	16
-- charge=0 and multiplicity=1 --
I     	    -3.63024893	    -3.68500590	     0.98014104
O     	    -6.52684861	     7.58716542	     1.12057174
H     	    -5.69339109	     7.81107291	     1.53739757
H     	    -5.92539387	     6.89614419	     0.83901956
O     	     4.70246001	    -5.76432958	     6.72580325
H     	     5.63981132	    -5.58367001	     6.64057164
H     	     4.74596752	    -4.98120933	     6.17502239
O     	    -7.39165746	     4.89912575	    -4.57856786
H     	    -7.73677825	     5.69764253	    -4.17633364
H     	    -6.71067543	     5.55643722	    -4.72934879
O     	     8.73801127	     2.02178637	    -4.37120300
H     	     8.37633339	     1.89771284	    -5.25002304
H     	     8.74686574	     2.89801835	    -4.75936391
O     	    -0.34273365	     0.15368232	    -9.95811080
H     	     0.18335824	     0.95167540	   -10.02856753
H     	     0.38036035	    -0.08711262	   -10.53921548



In [16]:
# py3Dmol: a simple IPython/Jupyter widget to embed an interactive 3Dmol.js viewer in a notebook.
!pip install py3Dmol
import py3Dmol

In [17]:
# 20% to check visually that everything is inside
sr = w5.sphere_radius * 1.2

# sphere center
cx, cy, cz = w5.sphere_center
sc = {"x": cx, "y": cy, "z": cz}

# axes
x_axis = {'start': {'x': cx-sr, 'y':cy, 'z':cz}, 'end': {'x': cx+sr, 'y':cy, 'z':cz}}
y_axis = {'start': {'x': cx, 'y':cy-sr, 'z':cz}, 'end': {'x': cx, 'y':cy+sr, 'z':cz}}
z_axis = {'start': {'x': cx, 'y':cy, 'z':cz-sr}, 'end': {'x': cx, 'y':cy, 'z':cz+sr}}

# starting visualization
xyz_view = py3Dmol.view(width=700, height=500)#, linked=False, viewergrid=(2,2))
xyz_view.addModelsAsFrames(w5_xyz,'xyz')
# xyz_view.setStyle({'stick': {}})
xyz_view.setStyle({'sphere': {'radius': 0.8}})
xyz_view.addSphere({'center': sc, 
        'radius': sr, 
        'color' :'yellow',
        'alpha': 0.5,
        })

# animation base on several XYZ coordinates snapshots
xyz_view.animate({'loop': "forward", 'speed': 1, 'reps': 1})

# cartesian 3D axes
xyz_view.addLine(x_axis)
xyz_view.addLine(y_axis)
xyz_view.addLine(z_axis)

xyz_view.addLabel("x", {
        'position':x_axis["end"],
        'inFront':'true',
        'fontSize':20,
        'showBackground':'false',
        'fontColor': 'black',
        })
xyz_view.addLabel("y", {
        'position':y_axis["end"],
        'inFront':'true',
        'fontSize':20,
        'showBackground':'false',
        'fontColor': 'black',
        })
xyz_view.addLabel("z", {
        'position':z_axis["end"],
        'inFront':'true',
        'fontSize':20,
        'showBackground':'false',
        'fontColor': 'black',
        })

xyz_view.zoomTo()
xyz_view.show()


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol